# The valley-Chern Insulator: A tight-binding simulation

Particles in honeycomb-like lattices such as graphene, hBN or TMDs possess a new valley degree of freedom, as the minima of the conduction band are located at the $K$ and $K′$ points instead of the Γ point of the Brillouin zone. If one engineers a interface between phases with opposite Berry curvature in the same valley, topologically protected edge-states appear at this interface. Altought these edge-states are only weakly protected by the crystalline structure of the lattice, they do not require time-reversal symmetry breaking, making them easy to engineer, especially in optical waveguide structures ([ref](doi.org/10.1103/PhysRevLett.99.236809)).

Here, we introduce the physics of this valley-Chern number, using the minimal model of the staggered honeycomb lattice, that can be described by the tight-binding Hamiltonian:

$$H_{\bm{k}} = \begin{bmatrix} \epsilon_0 /2  &  -t \gamma(\bm{k}) \\ -t \gamma^* (\bm{k}) &  -\epsilon_0 /2 \end{bmatrix},\:\:\: \gamma(\bm{k}) = e^{-i\bm{k} \cdot \bm{d}_{1}}+e^{-i\bm{k} \cdot \bm{d}_{2}}+e^{-i\bm{k} \cdot \bm{d}_{3}}$$

Let's build this Hamiltonian using our module.

## Building the Hamiltonian

In [13]:
import numpy as np
from tightbinding.geometry import Orbital, Site, Unitcell, Lattice
from tightbinding.hamiltonian import Hopping, Energy, Hamiltonianbuilder

# First we build the unit cell
a = 2.4 # inter-pillar distance
a1 = [-3**0.5/2 * a, 3/2 * a] # 1st lattice vector
a2 = [3**0.5/2 * a, 3/2 * a] # 2nd lattice vector

s = Orbital("s")
A = Site("A", [0, a/2], [s])
B = Site("B", [0,-a/2], [s])
UC = Unitcell("honeycomb", [a1,a2])
UC.add_site([A,B])

# The lattice is made of a single unit cell, with periodic boundary conditions (with phase-shift)
honeycomb = Lattice("honeycomb", UC, (True, True))
honeycomb.add_unitcell((0,0), update=True)
honeycomb.plot()


This is a very simple lattice to build. Now, let's contruct the Hamiltonian, we want to explore three dimensions: the reciprocal space $k_x$, $k_y$ and the on-site energy difference $\epsilon_0$.

In [14]:
# defining the parameters
n_k = 151
kl = 1.5

params = {
    "t": -1,
    "kx": np.linspace(-kl,kl, n_k),
    "ky": np.linspace(-kl,kl, n_k),
    "eps0": np.linspace(-1,1, 21)
}

onsite = Energy(honeycomb)
onsite.set_energy("-eps0/2", nsite='A')
onsite.set_energy("eps0/2", nsite='B')

links = [(0,0), (1,0), (0,1)]
couplings = [Hopping(honeycomb, "t", "A_s", "B_s", link) for link in links]

H = Hamiltonianbuilder(honeycomb, params, reciprocalcoords=["kx","ky"])
H.set_on_site_energies(onsite)
H.add_couplings(couplings)
H.build()
bands, vecs = H.eigh()

## Band structure and Bloch eigenvectors

Two general functions to investigate the band structure and eigenvector structure of a Bloch Hamiltonian can be found in the "plotting" submodule.

In [15]:
from tightbinding.plotting import plot_bands, plot_eigenvectors

plot_bands(bands, "kx")


:DynamicMap   [eps0,ky]
   :NdOverlay   [band]
      :Curve   [kx]   (energy)

In [16]:
from tightbinding.utils import angle, conjugate

relative_phase = angle(conjugate(vecs.sel(component = 0))*vecs.sel(component = 1))
relative_amplitude = abs(vecs.sel(component = 1))**2 - abs(vecs.sel(component = 0))**2

plot_eigenvectors(
    plots = [
        [angle(vecs), abs(vecs), relative_amplitude],
        [vecs.real, vecs.imag, relative_phase]
        ], 
    eigvas=[
        [bands, bands, bands],
        [bands,bands, bands]
        ],
    templates=[
        ['phase', 'amplitude', 'symmetric'], 
        ['symmetric', 'symmetric', 'phase']
        ],
    titles = [
        ['phase', 'amplitude', 'S3'],
        ['Real part', 'Imaginary part', 'ϕ']
        ]
    )



    'data': [{'colorbar': {'len': 0.35,
                           'ticktext': […

'plot_eigenvectors' takes a list of list of data fields, eigenvalue maps, templates and titles, and return an array of plots, each being interactivly controlled by sliders. The templates can either be strings calling specific colormaps and ranges, or a dictionary containing the specificatiosn of the plot. See function doc for more details.

## Berry curvature

We can now compute the Berry curvature of this system by using the function 'berry_curvature' from the 'topology' submodule. It uses the 4-point formula
$$B^n(\bm{k}) = \arg \left[\braket{u^n_1|u^n_2}\braket{u^n_2|u^n_3}\braket{u^n_3|u^n_4}\braket{u^n_4|u^n_1}\right]$$

with the $u^n_i$ forming a small square around each k point.

In [17]:
from tightbinding.topology import berry_curvature

berry = berry_curvature(vecs)

plot_eigenvectors([[berry.sel(band=0), berry.sel(band=1)]], [['symmetric', 'symmetric']], [['B0', 'B1']], [[bands.sel(band=0), bands.sel(band=1)]])

    'data': [{'colorbar': {'len': 0.7, 'tickformat': '.2f', 'x': np.float64(0.42…

The Berry curvature forms poles of opposites signs in each valley of the Brillouin zone. These poles are spreading out when $\epsilon_0$ increases, and change signs when $\epsilon_0$ changes sign. A Valley-Chern insulator is formed when two phases with opposites valley-Chern numbers are placed side by side. Let's simulate this.

## Edge simulation

We first create a new lattice, simulating a staggered honeycomb nanotube:

In [18]:
# First we build the unit cell
a = 2.4 # inter-pillar distance
a1 = [3**0.5 * a, 0] # 1st lattice vector
a2 = [3**0.5/2 * a, 3/2 * a] # 2nd lattice vector

s = Orbital("s")
A = Site("A", [0, a/2], [s])
B = Site("B", [0,-a/2], [s])
UC = Unitcell("honeycomb", [a1,a2])
UC.add_site([A,B])

honeycomb = Lattice("honeycomb", UC, (True, True))
honeycomb.create_rectangle_lattice((1,20))
honeycomb.plot()


In [19]:
# defining the parameters
n_k = 151
kl = 1.5
params = {
    "t": -1,
    "kx": np.linspace(-kl,kl, n_k),
    "ky": 0,
    "eps0": np.linspace(-1,1, 21)
}

onsite = Energy(honeycomb)
onsite.set_energy("-eps0/2", nsite='A')
onsite.set_energy("eps0/2", nsite='B')

links = [(0,0), (0,1), (-1,0)]
couplings = [Hopping(honeycomb, "t", "A_s", "B_s", link) for link in links]

H = Hamiltonianbuilder(honeycomb, params, reciprocalcoords=["kx", 'ky'])
H.set_on_site_energies(onsite)
H.add_couplings(couplings)
H.build()

H.plot_onsite_energy(orbital = 's') # showing the on-site energies


    'data': [{'hoverinfo': 'all',
              'marker': {'cmid': 0,
          …

In [8]:

bands, vecs = H.eigh()

plot_bands(bands, x = 'kx')

:DynamicMap   [eps0]
   :NdOverlay   [band]
      :Curve   [kx]   (energy)

We get a gapped dispersion (expect for $\epsilon_0 = 0$), which is expected since the only phase we have is gapped phase. Now, let's flip the on_site energy difference for half the ribbon. This will create two interfaces with opposite valley chern numbers.

In [9]:
onsite.set_energy("eps0/2", nsite='A', condition='index[1]<10')
onsite.set_energy("-eps0/2", nsite='B', condition='index[1]<10')


H = Hamiltonianbuilder(honeycomb, params, reciprocalcoords=["kx", 'ky'])
H.set_on_site_energies(onsite)
H.add_couplings(couplings)
H.build()

H.plot_onsite_energy(orbital = 's') # showing the on-site energies


    'data': [{'hoverinfo': 'all',
              'marker': {'cmid': 0,
          …

In [10]:
bands, vecs = H.eigh()

plot_bands(bands, x = 'kx')

:DynamicMap   [eps0]
   :NdOverlay   [band]
      :Curve   [kx]   (energy)

Edge states have appeared in the gap, which is indicative of a valley-Chern insulator. Finally, let's see how the wavefunction of these edge states look:

In [11]:
honeycomb.plot_field(abs(vecs), orbital="s", vtype='amplitude', figargs={'width':500, 'height':700})

    'data': [{'hoverinfo': 'all',
              'marker': {'color': {'bdata': ('…

One can clearly see, for $|\vec{k}| \approx 0.78$, two interface states (for band index 19 and 20), which get more localized as $\epsilon_0$ increases.